# Prototype for applying the algorithm



## Pool of candidates

In [ ]:
import random
import pandas as pd

In [ ]:
df_orig = pd.read_csv('pool_of_candidates.csv', sep=";")
df = df_orig.copy()
df = df.reset_index()
df['Criminal record'] = df['Criminal record'] * 100
df['Ethnical minority'] = df['Ethnical minority'] * 100
df['Disability'] = df['Disability'] * 100
df['# children'] = df['# children'] * (100/df['# children'].max())
df['Work experience'] = df['Work experience'] * (100/df['Work experience'].max())
df['years of studies'] = (100*(df['years of studies'] - df['years of studies'].min()))/(df['years of studies'].max() - df['years of studies'].min())
df['University Rank'] = (100*(df['University Rank'] - df['University Rank'].min()))/(df['University Rank'].max() - df['University Rank'].min())
df['gender'] = df['gender'] * 100
candidate_list = [dict(df.loc[i,:]) for i in range(len(df.index))]
df.head()

,level_0,index,gender,University Rank,years of studies,GPA,French,English,Work experience,Communication skills,# children,Disability,Ethnical minority,Criminal record
0,0,0,0,65.263158,75.0,61,65,58,43.75,67,0.000000,100,0,0
1,1,1,0,75.789474,25.0,100,84,60,0.00,87,66.666667,0,0,0
2,2,2,0,18.421053,50.0,86,91,79,68.75,87,100.000000,100,0,0
3,3,3,0,89.473684,50.0,85,70,78,43.75,74,0.000000,0,100,0
4,4,4,0,8.421053,50.0,65,61,84,31.25,90,66.666667,0,100,0


We instantiate some candidates characterized by a given list of criteria. 

## Model for the hiring

We fix the weights of the model used for hiring to some arbitrary values. In the game the values are determined by the algorithm tangibly build by all the teams. 

In [ ]:
# we fix only the weights for some criteria, meaning that not all the criteria are considered by the model for hiring
weights = {
  'Communication skills': 
}

We define the function computing the score of all the candidates in the pool. 

In [ ]:
def score_function(candidate, weights):
  res = 0
  for key in weights.keys():
    subres = 1
    subweights = key.split('-')
    for subweight in subweights:
      if subweight[0] == "~":
        subres *= (100 - candidate[subweight[1:]])
      else:
        subres *= candidate[subweight]
    subres = subres**(1/len(subweights))
    subres *= weights[key]
    res += subres
  return res

In [ ]:
def remove_candidates(all_cand, ids_to_remove):
    res = []
    for cand in all_cand:
        if cand['index'] not in ids_to_remove:
            res.append(cand)
    return res

We apply the model on the pool of candidates and we select the first two better candidates. 

In [ ]:
weights_1 = {
  'English-Work experience': 3,
  'English-Work experience-GPA': 2,
  'English-Work experience-~GPA': 1,
  'English-Communication skills-Criminal record': 1,
  'English-Communication skills-~Criminal record': 1,
  'English-~Communication skills': 1
}

In [ ]:
team_1_list = remove_candidates(candidate_list, [1,2,4,6,11,14,17,29,36,39])
random.shuffle(team_1_list)
team_1_candidates = sorted(team_1_list, key=lambda x: score_function(x, weights_1), reverse=True)
"You get candidates no.: " + ", ".join([str(candidate['index']) for candidate in team_1_candidates[:5]])

'You get candidates no.: 35.0, 19.0, 32.0, 3.0, 26.0'

In [ ]:
weights_2 = {
  '~University Rank-~Communication skills-English': 2,
  '~University Rank-Communication skills-years of studies': 3,
  '~University Rank-Communication skills-~years of studies': 4
}

In [ ]:
team_2_list = remove_candidates(candidate_list, [1,2,4,6,11,18,22,27,35,38])
random.shuffle(team_2_list)
team_2_candidates = sorted(team_2_list, key=lambda x: score_function(x, weights_2), reverse=True)
"You get candidates no.: " + ", ".join([str(candidate['index']) for candidate in team_2_candidates[:5]])

'You get candidates no.: 33.0, 28.0, 29.0, 32.0, 5.0'

We print the score for each candidate. 

In [ ]:
for candidate in team_2_candidates:
  print(score_function(candidate, weights_1))

370.0
367.0
361.0
361.0
361.0
352.0
352.0
349.0
346.0
343.0
340.0
337.0
337.0
334.0
325.0
325.0
325.0
325.0
322.0
316.0
316.0
310.0
310.0
307.0
301.0
301.0
292.0
289.0
286.0
283.0
283.0
262.0
247.0
247.0
238.0
235.0
223.0
220.0
